# Forecaster Parameter Optimization

Use this notebook to explore different forecasting configurations that achieve a 30-minute horizon while minimising error metrics.

The workflow mirrors the existing forecasting notebook but adds a small parameter search utility.

## 1. Imports and Logging

In [35]:
import os

import sys

from pathlib import Path

import itertools

import logging



import numpy as np

import pandas as pd

from IPython.display import display



# Make sure local forecast modules are importable

sys.path.insert(0, str(Path.cwd()))



from prometheus_query import PrometheusClient

from arima_forecaster import ARIMAForecaster



logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

logger = logging.getLogger('parameter_search')



print("✓ Imports loaded")


✓ Imports loaded


## 2. Configure Prometheus Connection

In [36]:
PROMETHEUS_URL = os.getenv('PROMETHEUS_URL', 'http://host.docker.internal:9090')
prom_client = PrometheusClient(base_url=PROMETHEUS_URL, timeout=20.0)

if prom_client.is_healthy():
    print(f"✓ Connected to Prometheus at {PROMETHEUS_URL}")
else:
    print(f"⚠ Unable to reach Prometheus at {PROMETHEUS_URL}")
    print("  Make sure 'docker compose up -d' is running in docker_setup/")

✓ Connected to Prometheus at http://host.docker.internal:9090


## 3. Discover Metrics and Select Series

In [37]:
metrics = prom_client.discover_metrics('interface')
print(f"Found {len(metrics)} metrics starting with 'interface'")
if metrics:
    for metric_name in metrics[:10]:
        print(f"  - {metric_name}")
    if len(metrics) > 10:
        print(f"  ... and {len(metrics) - 10} more")
else:
    print("⚠ No interface metrics detected. Update the prefix if needed.")

if 'interface_recv_octets' in metrics:
    TARGET_METRIC = 'interface_recv_octets'
elif metrics:
    TARGET_METRIC = metrics[0]
else:
    TARGET_METRIC = 'interface_recv_octets'

print(f"Target metric: {TARGET_METRIC}")

series_candidates = prom_client.list_series(TARGET_METRIC, lookback_minutes=180)
print(f"Found {len(series_candidates)} candidate time series")
if series_candidates:
    for idx, candidate in enumerate(series_candidates[:5], start=1):
        labels_str = ', '.join(f"{k}={v}" for k, v in candidate.items() if k != '__name__')
        print(f"  {idx}. {{{labels_str}}}")
    if len(series_candidates) > 5:
        print(f"  ... and {len(series_candidates) - 5} more")
else:
    print("⚠ No labelled series detected. Using static fallback labels.")

Found 4 metrics starting with 'interface'
  - interface_recv_octets
  - interface_recv_packets
  - interface_send_octets
  - interface_send_packets
Target metric: interface_recv_octets
Found 14 candidate time series
  1. {hostname=edge1, instance=host.docker.internal:19100, interface_name=ifNotify_ens2f0, job=udp-receiver}
  2. {hostname=edge1, instance=host.docker.internal:19100, interface_name=ifNotify_ens2f1, job=udp-receiver}
  3. {hostname=edge1, instance=host.docker.internal:19100, interface_name=nni0_in, job=udp-receiver}
  4. {hostname=edge1, instance=host.docker.internal:19100, interface_name=nni0_out, job=udp-receiver}
  5. {hostname=edge1, instance=host.docker.internal:19100, interface_name=nni1_in, job=udp-receiver}
  ... and 9 more


## 4. Fetch Historical Data

In [38]:
if series_candidates:
    SELECTED_LABELS = {k: v for k, v in series_candidates[6].items() if k != '__name__'}
else:
    SELECTED_LABELS = {
        'hostname': 'edge1',
        'interface_name': 'ens2f0',
        'instance': 'host.docker.internal:19100',
        'job': 'udp-receiver',
    }
    print("Using fallback labels; adjust as needed.")

QUERY_LOOKBACK_MINUTES = 200
TARGET_HORIZON_MINUTES = 30
STEP_SECONDS = 5

history_df = prom_client.get_metric_history(
    metric=TARGET_METRIC,
    labels=SELECTED_LABELS,
    lookback_minutes=QUERY_LOOKBACK_MINUTES,
    step_seconds=STEP_SECONDS,
    horizon_minutes=TARGET_HORIZON_MINUTES,
)

print(f"Retrieved {len(history_df)} samples spanning {history_df['timestamp'].min()} to {history_df['timestamp'].max()}")
display(history_df.head())

Retrieved 5504 samples spanning 2025-10-27 13:56:54+00:00 to 2025-10-27 21:36:54+00:00


,timestamp,value
0,2025-10-27 13:56:54+00:00,299116.0
1,2025-10-27 13:56:59+00:00,299746.0
2,2025-10-27 13:57:04+00:00,300180.0
3,2025-10-27 13:57:09+00:00,300768.0
4,2025-10-27 13:57:14+00:00,301062.0


## 5. Define Search Space

In [39]:
LOOKBACK_OPTIONS = [60, 120, 150, 180]  # minutes
CADENCE_OPTIONS = [5, 10]  # seconds
ARIMA_ORDERS = [(2, 1, 2), (0, 1, 1), (1, 1, 0), (1, 1, 1)]

def stringify_order(order_tuple):
    return ','.join(str(part) for part in order_tuple)

param_grid = []
for lookback, cadence, order in itertools.product(LOOKBACK_OPTIONS, CADENCE_OPTIONS, ARIMA_ORDERS):
    if lookback <= TARGET_HORIZON_MINUTES:
        continue
    param_grid.append({
        'FORECAST_LOOKBACK_MINUTES': lookback,
        'FORECAST_HORIZON_MINUTES': TARGET_HORIZON_MINUTES,
        'FORECAST_CADENCE_SECONDS': cadence,
        'FORECAST_ARIMA_ORDER': stringify_order(order),
        'order_tuple': order,
    })

print(f"Prepared {len(param_grid)} parameter combinations")

Prepared 32 parameter combinations


## 6. Evaluation Helpers

In [40]:
TEMP_MODEL_ROOT = Path.cwd() / 'tmp' / 'parameter_search_models'
TEMP_MODEL_ROOT.mkdir(parents=True, exist_ok=True)

def push_env(overrides):
    previous = {key: os.environ.get(key) for key in overrides}
    for key, value in overrides.items():
        os.environ[key] = str(value)
    return previous

def pop_env(previous):
    for key, value in previous.items():
        if value is None:
            os.environ.pop(key, None)

        else:
            os.environ[key] = value

def evaluate_parameters(history, metric, labels, combinations):
    results = []
    for idx, config in enumerate(combinations, start=1):
        print(f"[{idx}/{len(combinations)}] Evaluating {config['FORECAST_ARIMA_ORDER']} (lookback={config['FORECAST_LOOKBACK_MINUTES']} min, cadence={config['FORECAST_CADENCE_SECONDS']} s)")
        overrides = {
            'FORECAST_LOOKBACK_MINUTES': config['FORECAST_LOOKBACK_MINUTES'],
            'FORECAST_HORIZON_MINUTES': config['FORECAST_HORIZON_MINUTES'],
            'FORECAST_CADENCE_SECONDS': config['FORECAST_CADENCE_SECONDS'],
            'FORECAST_ARIMA_ORDER': config['FORECAST_ARIMA_ORDER'],
            'FORECAST_ARIMA_FALLBACKS': '1,1,1;1,1,0;0,1,1',
            'FORECAST_MODEL_DIR': TEMP_MODEL_ROOT.joinpath(f"run_{idx:03d}").as_posix(),
        }
        previous = push_env(overrides)
        try:
            forecaster = ARIMAForecaster()
            required_points = forecaster.lookback_points() + forecaster.horizon_points()
            if len(history) < required_points:
                raise ValueError(f"Need >= {required_points} points, have {len(history)}")
            training_result = forecaster.train(metric=metric, labels=labels, history=history)
            results.append({
                'idx': idx,
                'lookback_minutes': config['FORECAST_LOOKBACK_MINUTES'],
                'horizon_minutes': config['FORECAST_HORIZON_MINUTES'],
                'cadence_seconds': config['FORECAST_CADENCE_SECONDS'],
                'arima_order': training_result.get('arima_order', config['order_tuple']),
                'rmse': training_result.get('rmse', float('nan')),
                'residual_std': training_result.get('residual_std', float('nan')),
                'baseline_weight': training_result.get('baseline_weight', float('nan')),
                'samples': training_result.get('samples', len(history)),
                'promoted': training_result.get('promoted', False),
                'error': None,
            })
        except Exception as exc:
            logger.warning('Configuration failed', exc_info=True)
            results.append({
                'idx': idx,
                'lookback_minutes': config['FORECAST_LOOKBACK_MINUTES'],
                'horizon_minutes': config['FORECAST_HORIZON_MINUTES'],
                'cadence_seconds': config['FORECAST_CADENCE_SECONDS'],
                'arima_order': config['order_tuple'],
                'rmse': np.nan,
                'residual_std': np.nan,
                'baseline_weight': np.nan,
                'samples': len(history),
                'promoted': False,
                'error': str(exc),
            })
        finally:
            pop_env(previous)
    return pd.DataFrame(results)

## 7. Run Parameter Search

In [41]:
if history_df.empty:
    raise RuntimeError('No history available. Adjust metric/labels or ensure Prometheus has data.')

search_results = evaluate_parameters(history_df, TARGET_METRIC, SELECTED_LABELS, param_grid)
search_results

2025-10-27 22:37:02,856 - arima_forecaster - INFO - ARIMAForecaster initialised (order=(2, 1, 2) lookback=60min horizon=30min cadence=5sec models=/home/peti/networking-dashboard/app/forecast/tmp/parameter_search_models/run_001)
2025-10-27 22:37:02,857 - arima_forecaster - INFO - Training ARIMA model for interface_recv_octets/d27a8c185e43ff5d (samples=5504)


[1/32] Evaluating 2,1,2 (lookback=60 min, cadence=5 s)


2025-10-27 22:37:07,105 - arima_forecaster - INFO - Training completed for interface_recv_octets/d27a8c185e43ff5d version=v20251027-2137 rmse=455.5746 baseline_w=0.00 promoted=True
2025-10-27 22:37:07,106 - arima_forecaster - INFO - ARIMAForecaster initialised (order=(0, 1, 1) lookback=60min horizon=30min cadence=5sec models=/home/peti/networking-dashboard/app/forecast/tmp/parameter_search_models/run_002)
2025-10-27 22:37:07,106 - arima_forecaster - INFO - Training ARIMA model for interface_recv_octets/d27a8c185e43ff5d (samples=5504)


[2/32] Evaluating 0,1,1 (lookback=60 min, cadence=5 s)


2025-10-27 22:37:07,752 - arima_forecaster - INFO - Training completed for interface_recv_octets/d27a8c185e43ff5d version=v20251027-2137 rmse=47286.3477 baseline_w=0.50 promoted=True
2025-10-27 22:37:07,755 - arima_forecaster - INFO - ARIMAForecaster initialised (order=(1, 1, 0) lookback=60min horizon=30min cadence=5sec models=/home/peti/networking-dashboard/app/forecast/tmp/parameter_search_models/run_003)
2025-10-27 22:37:07,756 - arima_forecaster - INFO - Training ARIMA model for interface_recv_octets/d27a8c185e43ff5d (samples=5504)
2025-10-27 22:37:07,909 - arima_forecaster - WARNING - ARIMA fit failed for order (1, 1, 0): ARIMA order (1, 1, 0) failed to converge (maxiter=200)


...

[30/32] Evaluating 0,1,1 (lookback=180 min, cadence=10 s)


2025-10-27 22:38:14,897 - arima_forecaster - INFO - Training completed for interface_recv_octets/d27a8c185e43ff5d version=v20251027-2138 rmse=47284.1094 baseline_w=0.50 promoted=True
2025-10-27 22:38:14,901 - arima_forecaster - INFO - ARIMAForecaster initialised (order=(1, 1, 0) lookback=180min horizon=30min cadence=10sec models=/home/peti/networking-dashboard/app/forecast/tmp/parameter_search_models/run_031)
2025-10-27 22:38:14,902 - arima_forecaster - INFO - Training ARIMA model for interface_recv_octets/d27a8c185e43ff5d (samples=5504)
2025-10-27 22:38:15,049 - arima_forecaster - INFO - Training completed for interface_recv_octets/d27a8c185e43ff5d version=v20251027-2138 rmse=37903.5039 baseline_w=0.25 promoted=True
2025-10-27 22:38:15,051 - arima_forecaster - INFO - ARIMAForecaster initialised (order=(1, 1, 1) lookback=180min horizon=30min cadence=10sec models=/home/peti/networking-dashboard/app/forecast/tmp/parameter_search_models/run_032)
2025-10-27 22:38:15,051 - arima_forecaster 

[31/32] Evaluating 1,1,0 (lookback=180 min, cadence=10 s)
[32/32] Evaluating 1,1,1 (lookback=180 min, cadence=10 s)


2025-10-27 22:38:15,692 - arima_forecaster - WARNING - ARIMA fit failed for order (1, 1, 1): ARIMA order (1, 1, 1) failed to converge (maxiter=200)
2025-10-27 22:38:15,758 - arima_forecaster - INFO - Using fallback ARIMA order (1, 1, 0) (series=2581)
2025-10-27 22:38:15,833 - arima_forecaster - INFO - Training completed for interface_recv_octets/d27a8c185e43ff5d version=v20251027-2138 rmse=37903.5039 baseline_w=0.25 promoted=True


,idx,lookback_minutes,horizon_minutes,cadence_seconds,arima_order,rmse,residual_std,baseline_weight,samples,promoted,error
0,1,60,30,5,"(2, 1, 2)",455.574646,240.669189,0.00,5521,True,None
1,2,60,30,5,"(0, 1, 1)",47286.347656,47285.550781,0.50,5521,True,None
2,3,60,30,5,"(1, 1, 1)",526.770508,257.721436,0.00,5521,True,None
3,4,60,30,5,"(1, 1, 1)",526.770508,257.721436,0.00,5521,True,None
4,5,60,30,10,"(2, 1, 2)",152.189606,97.612480,0.00,2761,True,None
5,6,60,30,10,"(0, 1, 1)",47284.109375,47283.156250,0.50,2761,True,None
6,7,60,30,10,"(1, 1, 0)",37903.503906,36276.664062,0.25,2761,True,None
7,8,60,30,10,"(1, 1, 0)",37903.503906,36276.664062,0.25,2761,True,None
8,9,120,30,5,"(2, 1, 2)",455.574646,240.669189,0.00,5521,True,None
9,10,120,30,5,"(0, 1, 1)",47286.347656,47285.550781,0.50,5521,True,None


## 8. Review Best Configurations

In [42]:
successful = search_results.dropna(subset=['rmse']).sort_values('rmse')
if successful.empty:
    print('⚠ No successful runs. Check error messages above.')
else:
    print('Top 5 configurations by RMSE:')
    display(successful.head(5))
    best_config = successful.iloc[0]
    print('Best configuration (for 30 min horizon):')
    for field in ['lookback_minutes', 'horizon_minutes', 'cadence_seconds', 'arima_order', 'rmse']:
        print(f"  {field}: {best_config[field]}")
    print(f"  baseline_weight: {best_config['baseline_weight']}")
    print(f"  samples: {int(best_config['samples'])}")

Top 5 configurations by RMSE:


,idx,lookback_minutes,horizon_minutes,cadence_seconds,arima_order,rmse,residual_std,baseline_weight,samples,promoted,error
4,5,60,30,10,"(2, 1, 2)",152.189606,97.612480,0.0,2761,True,None
12,13,120,30,10,"(2, 1, 2)",152.189606,97.612480,0.0,2761,True,None
28,29,180,30,10,"(2, 1, 2)",152.189606,97.612480,0.0,2761,True,None
20,21,150,30,10,"(2, 1, 2)",152.189606,97.612480,0.0,2761,True,None
16,17,150,30,5,"(2, 1, 2)",455.574646,240.669189,0.0,5521,True,None


Best configuration (for 30 min horizon):
  lookback_minutes: 60
  horizon_minutes: 30
  cadence_seconds: 10
  arima_order: (2, 1, 2)
  rmse: 152.18960571289062
  baseline_weight: 0.0
  samples: 2761


## 9. Apply Best Configuration (Optional)

In [43]:
if 'best_config' in globals():
    overrides = {
        'FORECAST_LOOKBACK_MINUTES': int(best_config['lookback_minutes']),
        'FORECAST_HORIZON_MINUTES': TARGET_HORIZON_MINUTES,
        'FORECAST_CADENCE_SECONDS': int(best_config['cadence_seconds']),
        'FORECAST_ARIMA_ORDER': ','.join(str(v) for v in best_config['arima_order']),
    }
    print('Suggested environment overrides:')
    for key, value in overrides.items():
        print(f"export {key}={value}")
else:
    print('Run the search first to populate best_config.')

Suggested environment overrides:
export FORECAST_LOOKBACK_MINUTES=60
export FORECAST_HORIZON_MINUTES=30
export FORECAST_CADENCE_SECONDS=10
export FORECAST_ARIMA_ORDER=2,1,2
